# Training YOLOv3 on Fabric Stain Dataset

This notebook demonstrates how to train a YOLOv3 model on a custom dataset of fabric stains. The dataset consists of images stored in `.\\fabric_stain_dataset/split_images` and annotations in `.\\fabric_stain_dataset/annotations`.

In [7]:
# Install necessary libraries
!pip install tensorflow
!pip install opencv-python
!pip install lxml
!pip install matplotlib
!pip install PyYAML
!pip install tqdm
!pip install absl-py
!pip install tensorflow-addons


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement tensorflow-addons (from versions: none)
ERROR: No matching distribution found for tensorflow-addons

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import Libraries

In [30]:
import tensorflow as tf
from keras import layers, models
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import yaml
from absl import app, flags, logging
from absl.flags import FLAGS
from PIL import Image

## Load Dataset

In [35]:
import os
import cv2
import numpy as np


def load_images_and_labels(image_dir, annotation_dir):
    images = []
    labels = []
    max_label_length = 0
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(image_dir, filename)
            annotation_path = os.path.join(annotation_dir, filename.replace('.jpg', '.txt').replace('.png', '.txt'))
            image = Image.open(image_path)
            image = np.array(image) / 255.0 # Convert the image to a numpy array and normalize it
            images.append(image)
            with open(annotation_path, 'r') as file:
                label = []
                for line in file:
                    label.append([float(x) for x in line.strip().split()])
                labels.append(label)
                if len(label) > max_label_length:
                    max_label_length = len(label)
    
    # Pad labels to ensure they all have the same length
    for label in labels:
        while len(label) < max_label_length:
            label.append([0.0] * len(label[0]))
    
    return np.array(images), np.array(labels) # Return images and labels as lists


image_dir = '.\\fabric_stain_dataset\\images\\stain'
annotation_dir = '.\\fabric_stain_dataset\\annotations\\stain'

images, labels = load_images_and_labels(image_dir, annotation_dir)

print(images)
print(labels)


print(f'Loaded {len(images)} images and {len(labels)} labels')

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (398,) + inhomogeneous part.

## Define YOLOv3 Model

In [ ]:
def yolo_v3_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    # Define the YOLOv3 architecture here
    # This is a simplified version for demonstration purposes
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    outputs = layers.Dense(5, activation='sigmoid')(x)  # Assuming 5 classes
    model = models.Model(inputs, outputs)
    return model

input_shape = (32, 32, 1)  # Update input shape for 32x32 grayscale images
model = yolo_v3_model(input_shape)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32, 32, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 30, 30, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 357,893 (1.37 MB)

 Trainable params: 357,893 (1.37 MB)

 Non-trainable params: 0 (0.00 B)

## Compile and Train the Model

In [25]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming images and labels are preprocessed and ready for training
history = model.fit(images, labels, epochs=10, batch_size=8, validation_split=0.2)

ValueError: Argument `validation_split` is only supported for tensors or NumPy arrays.Found incompatible type in the input: [<class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class '

## Evaluate the Model

In [ ]:
loss, accuracy = model.evaluate(images, labels)
print(f'Loss: {loss}, Accuracy: {accuracy}')

## Visualize Training History

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()